In [1]:
import faiss
from faiss import write_index, read_index


In [2]:
import numpy as np
import pandas as pd
import os
import time

In [3]:
filename = '.\data\encoded\encoded_transcript_01.csv'
df = pd.read_csv(filename,index_col=0)
df.head()

,text,start_time,stop_time,embeddings
0,จุดจุดจุด อ่ะก่อนอื่น คนเรือง เลือดสำคัญต่อก็ค...,2023-12-25 0:00:04,2023-12-25 0:01:13,[ 3.20836790e-02 8.82582888e-02 -1.68851633e-...
1,คือเราวิชาแบกในสามส่วนแล้วมันก็ตอบแล้วมันก็สลา...,2023-12-25 0:01:15,2023-12-25 0:02:09,[ 6.50551766e-02 1.55494493e-02 -1.16257565e-...
2,ภายนอกก็คือจะตอบส่วนสไตล์วิชา เอ่อ น้อง อ่านหน...,2023-12-25 0:02:10,2023-12-25 0:03:17,[ 3.14746313e-02 1.31374389e-01 -1.45527218e-...
3,มี tina อย่างเดียว สองเจ็ด สลายสามารถโดนความรู...,2023-12-25 0:03:17,2023-12-25 0:04:14,[ 6.46190867e-02 9.32341069e-02 -1.23647125e-...
4,โอ๊ตลูกแตะลูกอาจารย์อิรักสักคนนึงแปะมาขนาด เอ่...,2023-12-25 0:04:17,2023-12-25 0:05:09,[ 1.27054388e-02 -1.97878495e-01 -1.13094635e-...


In [4]:
df['embeddings'] = df['embeddings'].apply(lambda x:np.array([float(a) for a in x[2:-1].split()]))

In [5]:
from sentence_transformers import SentenceTransformer

In [6]:
from sentence_transformers import SentenceTransformer

model_name = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
max_seq_length = 128

# Create the model with CPU
# model = SentenceTransformer(model_name, device='cpu')
model = SentenceTransformer(model_name, device='cuda')
model.max_seq_length = max_seq_length


In [7]:
d = 768
i=1
for file in os.listdir(f'./data/labeled'):
    print(file)
    if "transcript" not in file:
        continue
    filename = f"./data/labeled/{file}"
    df = pd.read_csv(filename, index_col=0)
    corpus_embeddings = df['text']
    corpus_embeddings = model.encode(
        corpus_embeddings, convert_to_tensor=False, show_progress_bar=True
        )
    df['embeddings'] = list(corpus_embeddings)
    index = faiss.IndexFlatL2(d)
    index.add(np.stack(df['embeddings'].to_numpy()))
    write_index(index, f"./data/faiss/t{i}.index")
    i+=1


file_query_map.csv
labeled_transcript_01.csv


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

labeled_transcript_02.csv


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

labeled_transcript_03.csv


...

Batches:   0%|          | 0/6 [00:00<?, ?it/s]

labeled_transcript_08.csv


Batches:   0%|          | 0/5 [00:00<?, ?it/s]

labeled_transcript_09.csv


Batches:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# d = 768
# start_time = time.time()
# index = faiss.IndexFlatL2(d)
# # print(index.is_trained)
# index.add(np.stack(df['embeddings'].to_numpy()))
# end_time = time.time()
# print(end_time-start_time)
# print(index.ntotal)

In [9]:
model.to("cpu")
next(model.parameters()).is_cuda

False

In [10]:
file_query_map_df = pd.read_csv("./data/labeled/file_query_map.csv")
file_query_map_df.head()

,file,query_type,query
0,labeled_transcript_01.csv,keyword,linear program
1,labeled_transcript_01.csv,keyword,Assumption of linear program
2,labeled_transcript_01.csv,keyword,standard form of linear program
3,labeled_transcript_01.csv,keyword,basic feasible solution
4,labeled_transcript_01.csv,keyword,direction of unboundedness


In [11]:
search_time = []
for file in os.listdir(f'./data/labeled'):
    if "transcript" not in file:
        continue
    filename = f"./data/labeled/{file}"
    id = file[-5:-4]
    print(id)
    for query in file_query_map_df.loc[file_query_map_df.file==file, "query"].tolist():
        start_time = time.time_ns()
        query_embedding = model.encode([query], convert_to_tensor=False)
        index = read_index(f"./data/faiss/t{id}.index")
        
        k = 10
        D, I = index.search(query_embedding, k)
        end_time = time.time_ns()
        search_time.append((end_time-start_time)/1e9)

1
2
3
4
5
6
7
8
9


In [12]:
np.mean(search_time)

0.16197368444444443

In [14]:
# df = pd.read_csv(f"./data/labeled/labeled_transcript_02.csv", index_col=0)

In [44]:
search_time = []
fields = [
    "text",
    "start_time",
    "stop_time",
]


for file in os.listdir(f"./data/labeled"):
    if "transcript" not in file:
        continue
    filename = f"./data/labeled/{file}"
    id = file[-5:-4]
    print(id)
    for query in file_query_map_df.loc[
        file_query_map_df.file == file, "query"
    ].tolist():
        labeled_df = pd.read_csv(
            f"./data/labeled/labeled_transcript_0{id}.csv", index_col=0, usecols=fields
        )
        start_time = time.time_ns()
        query_embedding = model.encode([query], convert_to_tensor=False)
        index = read_index(f"./data/faiss/t{id}.index")
        k = 10
        D, I = index.search(query_embedding, k)


        search_index_list = I[0].tolist()
        search_result = labeled_df.iloc[search_index_list]
        search_result_start_time = search_result["start_time"]

        end_time = time.time_ns()
        search_time.append((end_time - start_time) / 1e9)

1
2
3
4
5
6
7
8
9


In [45]:
np.mean(search_time)

0.12775799888888886

In [47]:
x = [
0.14787364888888888,
0.1384574322222222,
0.13446970666666666,
0.10524345333333333,
0.12775799888888886,
]
np.mean(x)

0.13076044799999997

In [48]:
# mean search time (not map transcript) (sec) (avg=0.11621410822222225)
# 0.14588159333333334
# 0.11387941000000001
# 0.11385797555555559
# 0.11219363555555556
# 0.09525792666666667

# mean search time (map transcript, read dataframe every query) (sec) (avg=0.18186827533333333)
# 0.20160134555555556
# 0.1354372033333333
# 0.1557293311111111
# 0.2020292477777778
# 0.2145442488888889

# mean search time (map transcript, dataframe already in memory) (sec) (avg=0.13076044799999997)
# 0.14787364888888888
# 0.1384574322222222
# 0.13446970666666666
# 0.10524345333333333
# 0.12775799888888886